# CSV vs PARQUET (Arrow)

- format scalability
- using local DuckDB
- with MovieLens 100k and 33M


## Setup — Imports & Environment Configuration

In [1]:
# ===============================================
#   IMPORTS & PATHS & SETUP
# ===============================================

import duckdb
import pandas as pd
import polars as pl
import time
from pathlib import Path

# === PATHS FOR THE DATASETS ===

## MovieLens 100k
DATA_100k = Path("..") / "data" / "100K"
ratings_100k = DATA_100k / "ratings.csv"
movies_100k  = DATA_100k / "movies.csv"
tags_100k    = DATA_100k / "tags.csv"
links_100k   = DATA_100k / "links.csv"

## MovieLens 33M
DATA_33m = Path("..") / "data" / "Full33M"
ratings_33m = DATA_33m / "ratings.csv"
movies_33m  = DATA_33m / "movies.csv"
tags_33m    = DATA_33m / "tags.csv"
links_33m   = DATA_33m / "links.csv"

print("Paths defined successfully.")

# === Local DuckDB connection ===
con = duckdb.connect("movielens_local.duckdb")
print("DuckDB connection opened.")

Paths defined successfully.
DuckDB connection opened.


In [2]:
print(ratings_100k)
print(ratings_33m)

..\data\100K\ratings.csv
..\data\Full33M\ratings.csv


## Create Parquet Tables in DuckDB (ratings_parquet, movies_parquet)

In [3]:
# === CONVERSÃO CSV -> PARQUET  ===

ratings_100k_parquet = ratings_100k.with_suffix(".parquet")
ratings_33m_parquet  = ratings_33m.with_suffix(".parquet")
movies_33m_parquet   = movies_33m.with_suffix(".parquet")

print("Parquet paths:")
print(ratings_100k_parquet)
print(ratings_33m_parquet)
print(movies_33m_parquet)

# Converter apenas se precisares (simples: corre sempre para já)
duckdb.sql(f"""
COPY (SELECT * FROM read_csv_auto('{ratings_100k}'))
TO '{ratings_100k_parquet}'
(FORMAT PARQUET);
""")

duckdb.sql(f"""
COPY (SELECT * FROM read_csv_auto('{ratings_33m}'))
TO '{ratings_33m_parquet}'
(FORMAT PARQUET);
""")

duckdb.sql(f"""
COPY (SELECT * FROM read_csv_auto('{movies_33m}'))
TO '{movies_33m_parquet}'
(FORMAT PARQUET);
""")

print("CSV -> Parquet conversion finished.")


Parquet paths:
..\data\100K\ratings.parquet
..\data\Full33M\ratings.parquet
..\data\Full33M\movies.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CSV -> Parquet conversion finished.


In [ ]:
# === VIEWS DuckDB sobre os ficheiros Parquet ===

con.execute(f"""
    CREATE OR REPLACE VIEW ratings_100k_parquet AS
    SELECT * FROM read_parquet('{ratings_100k_parquet}')
""")

con.execute(f"""
    CREATE OR REPLACE VIEW ratings_33m_parquet AS
    SELECT * FROM read_parquet('{ratings_33m_parquet}')
""")

con.execute(f"""
    CREATE OR REPLACE VIEW movies_33m_parquet AS
    SELECT * FROM read_parquet('{movies_33m_parquet}')
""")

con.sql("SHOW TABLES").df()


,name
0,movies_33m_parquet
1,ratings_100k_parquet
2,ratings_33m_parquet
3,ratings_parquet
4,tags_parquet


In [7]:
# Função medir_tempo

def medir_tempo(query):
    t0 = time.time()
    duckdb.sql(query).df()
    return round(time.time() - t0, 3)


## Benchmark 1 — CSV vs Parquet Performance (AVG ratings + GROUP BY moviesId)

We measure the execution time of a simple aggregation query (`AVG(rating)` grouped by `movieId`) on CSV and Parquet, for both MovieLens 100k and 33M. This isolates the impact of the storage format.

In [8]:
# Medições CSV vs Parquet

tempos = {
    "CSV_100k": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_100k}'
        GROUP BY movieId
    """),

    "PARQUET_100k": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_100k.with_suffix('.parquet')}'
        GROUP BY movieId
    """),

    "CSV_33M": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_33m}'
        GROUP BY movieId
    """),

    "PARQUET_33M": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_33m.with_suffix('.parquet')}'
        GROUP BY movieId
    """),
}

tempos


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

{'CSV_100k': 0.078,
 'PARQUET_100k': 0.018,
 'CSV_33M': 2.207,
 'PARQUET_33M': 0.212}

In [9]:
# Tabela comparativa

df_tempo = pd.DataFrame([
    ["100k", tempos["CSV_100k"], tempos["PARQUET_100k"]],
    ["33M", tempos["CSV_33M"], tempos["PARQUET_33M"]],
], columns=["Dataset", "CSV time (s)", "Parquet time (s)"])

df_tempo


,Dataset,CSV time (s),Parquet time (s)
0,100k,0.078,0.018
1,33M,2.207,0.212


__Conclusion:__

This benchmark measures the execution time of a simple aggregation query (`AVG(rating)` grouped by `movieId`) across four scenarios:

- CSV 100K
- Parquet 100K
- CSV 33M
- Parquet 33M

The results show the strong impact of storage format on query speed, with Parquet providing substantial improvements—especially at larger scales.


__Close the connection (when done)__

In [23]:
#con.close()
#print("Connection closed.")

# DuckDB vs Polars

## Benchmark 2 — Simple COUNT() Query (DuckDB vs Polars)

Here we run the same aggregation query (`AVG(rating)` grouped by `movieId`) on Parquet files using DuckDB and Polars, for 100k and 33M rows, in order to compare the execution engines under a simple workload.

In [11]:
# paths para Parquet

p100k = ratings_100k.with_suffix(".parquet")
p33m = ratings_33m.with_suffix(".parquet")

In [12]:
# Funções de benchmark

def run_duckdb(path):
    t0 = time.time()
    duckdb.sql(f"""
        SELECT movieId, AVG(rating)
        FROM '{path}'
        GROUP BY movieId
    """).df()
    return round(time.time() - t0, 3)

def run_polars(path):
    t0 = time.time()
    (
        pl.scan_parquet(str(path))     
          .group_by("movieId")         
          .agg(pl.col("rating").mean())
          .collect()
    )
    return round(time.time() - t0, 3)


In [13]:
# Comparação

df_duck_polars = pd.DataFrame([
    ["100k", run_duckdb(p100k), run_polars(p100k)],
    ["33m", run_duckdb(p33m), run_polars(p33m)],
], columns=["Dataset", "DuckDB (s)", "Polars (s)"])

df_duck_polars


,Dataset,DuckDB (s),Polars (s)
0,100k,0.007,0.123
1,33m,0.171,0.605


## Benchmark 3 — Aggregation + JOIN (AVG, COUNT, STDDEV) — DuckDB vs Polars

This benchmark uses a more realistic analytical workload on the 33M dataset:
we compute average rating, number of ratings and standard deviation per movie, join with the movies table to retrieve titles, filter movies with at least 500 ratings, and sort by total_ratings (TOP 100). The goal is to compare DuckDB and Polars under a heavier analytical query.

In [14]:
MIN_RATINGS = 500

def run_duckdb_movie_stats(con):
    sql = f"""
    WITH movie_stats AS (
        SELECT
            m.movieId,
            m.title,
            AVG(r.rating) AS avg_rating,
            COUNT(*) AS total_ratings,
            STDDEV_POP(r.rating) AS std_rating
        FROM ratings_33m_parquet r
        JOIN movies_33m_parquet m USING (movieId)
        GROUP BY m.movieId, m.title
        HAVING COUNT(*) >= {MIN_RATINGS}
    )
    SELECT
        movieId,
        title,
        ROUND(avg_rating, 3)  AS avg_rating,
        total_ratings,
        ROUND(std_rating, 3)  AS std_rating
    FROM movie_stats
    ORDER BY total_ratings DESC
    LIMIT 100
    """
    return con.sql(sql).df()


In [27]:
start = time.perf_counter()
df_duckdb_stats = run_duckdb_movie_stats(con)
t_duckdb_stats = time.perf_counter() - start

#df_duckdb_stats.head(), t_duckdb_stats


In [28]:
import time

start = time.perf_counter()
df_duckdb_stats = run_duckdb_movie_stats(con)
t_duckdb_stats = time.perf_counter() - start

#df_duckdb_stats.head(), t_duckdb_stats


In [17]:
### Polars

In [20]:
def run_polars_movie_stats_33m():
    start = time.perf_counter()

    # usar os ficheiros PARQUET, não os CSV
    ratings = (
        pl.scan_parquet(str(ratings_33m_parquet))
        .select(["movieId", "rating"])
    )

    movies  = (
        pl.scan_parquet(str(movies_33m_parquet))
        .select(["movieId", "title"])
    )

    MIN_RATINGS = 500

    result = (
        ratings
        .group_by("movieId")
        .agg([
            pl.col("rating").mean().alias("avg_rating"),
            pl.count().alias("total_ratings"),
            pl.col("rating").std().alias("std_rating"),
        ])
        .filter(pl.col("total_ratings") >= MIN_RATINGS)
        .join(movies, on="movieId")
        .select([
            "movieId",
            "title",
            pl.col("avg_rating").round(3),
            "total_ratings",
            pl.col("std_rating").round(3),
        ])
        .sort("total_ratings", descending=True)
        .limit(100)
        .collect()
    )

    elapsed = time.perf_counter() - start
    return result, elapsed


In [30]:
#df_polars_stats_33m, t_polars_stats_33m = run_polars_movie_stats_33m()
#df_polars_stats_33m.head(), t_polars_stats_33m


In [25]:
df_complex = pd.DataFrame([
    ["33M (complex)", t_duckdb_stats, t_polars_stats_33m],
], columns=["Dataset", "DuckDB (s)", "Polars (s)"])

df_complex


,Dataset,DuckDB (s),Polars (s)
0,33M (complex),1.023147,0.676009


# Summary

In [ ]:
df_summary = pd.DataFrame([
    ["Benchmark 1 - 100k (CSV vs Parquet)", "DuckDB CSV", tempos["CSV_100k"]],
    ["Benchmark 1 - 100k (CSV vs Parquet)", "DuckDB Parquet", tempos["PARQUET_100k"]],
    ["Benchmark 1 - 33M (CSV vs Parquet)", "DuckDB CSV", tempos["CSV_33M"]],
    ["Benchmark 1 - 33M (CSV vs Parquet)", "DuckDB Parquet", tempos["PARQUET_33M"]],
    ["Benchmark 2 - 100k (simple)", "DuckDB", df_duck_polars.loc[0, "DuckDB (s)"]],
    ["Benchmark 2 - 100k (simple)", "Polars", df_duck_polars.loc[0, "Polars (s)"]],
    ["Benchmark 2 - 33M (simple)", "DuckDB", df_duck_polars.loc[1, "DuckDB (s)"]],
    ["Benchmark 2 - 33M (simple)", "Polars", df_duck_polars.loc[1, "Polars (s)"]],
    ["Benchmark 3 - 33M (complex)", "DuckDB", df_complex.loc[0, "DuckDB (s)"]],
    ["Benchmark 3 - 33M (complex)", "Polars", df_complex.loc[0, "Polars (s)"]],
], columns=["Scenario", "Engine", "Time (s)"])

df_summary


,Scenario,Engine,Time (s)
0,Benchmark 1 – 100k (CSV vs Parquet),DuckDB CSV,0.078000
1,Benchmark 1 – 100k (CSV vs Parquet),DuckDB Parquet,0.018000
2,Benchmark 1 – 33M (CSV vs Parquet),DuckDB CSV,2.207000
3,Benchmark 1 – 33M (CSV vs Parquet),DuckDB Parquet,0.212000
4,Benchmark 2 – 100k (simple),DuckDB,0.007000
5,Benchmark 2 – 100k (simple),Polars,0.123000
6,Benchmark 2 – 33M (simple),DuckDB,0.171000
7,Benchmark 2 – 33M (simple),Polars,0.605000
8,Benchmark 3 – 33M (complex),DuckDB,1.023147
9,Benchmark 3 – 33M (complex),Polars,0.676009


The results show that Parquet is substantially faster than CSV, delivering up to a considerable improvement on the 33M dataset.

In the engine comparison (Benchmark 2), DuckDB is faster on simple aggregations—especially on smaller datasets—while Polars remains competitive.

In the complex analytical query (Benchmark 3), Polars outperforms DuckDB (0.67s vs 1.02s), demonstrating better scalability under heavier workloads.

Both engines achieve sub-second performance on 33M rows when using Parquet, highlighting the efficiency of columnar storage.

Overall, Parquet + Polars provides the strongest performance for large-scale analytical queries.